In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = 999

### Scrape 2010-2019 data

In [3]:
url = 'https://www.sportsoddshistory.com/nba-regular-season-win-total-results-by-team/'
info = pd.read_html(url)
df = pd.DataFrame(info[0])
df = df.droplevel(0, axis=1)
df.drop(columns = ["1990's", "2000's", "2010's", "1990 – 2019", "Streak", "11-12*"], inplace = True)
# not including 2011-12 because it was a strike shortened 66 game season.  
df.head()

,Team,09-10,10-11,12-13,13-14,14-15,15-16,16-17,17-18,18-19
0,Atlanta Hawks,44.5 53 O,46.5 44 U,42.5 44 O,39.5 38 U,42.5 60 O,49.5 48 U,43.5 43 U,25.5 24 U,23.5 29 O
1,Boston Celtics,56.5 50 U,52.5 56 O,51.5 41 U,28.5 25 U,27 40 O,45.5 48 O,52.5 53 O,53.5 55 O,59 49 U
2,Brooklyn Nets,27.5 12 U,24.5 24 U,44.5 49 O,52.5 44 U,41.5 38 U,27.5 21 U,21.5 20 U,27.5 28 O,32 42 O
3,Charlotte Hornets,36.5 44 O,39.5 34 U,19.5 21 O,26 43 O,45 33 U,34 48 O,42.5 36 U,42.5 36 U,35.5 39 O
4,Chicago Bulls,40.5 41 O,46.5 62 O,47.5 45 U,56.5 48 U,55.5 50 U,50.5 42 U,39 41 O,22 27 O,30 22 U


In [4]:
lst = []
for i in range(df.shape[0]-1):
    for col in df.drop(columns = 'Team').columns:
        lst.append([f'{df.iloc[i][0]}_{col[-2:]}', df[col][i]])    

df1 = pd.DataFrame(lst)
df1.rename(columns = {0: 'Team'}, inplace = True)
df1.head()

,Team,1
0,Atlanta Hawks_10,44.5 53 O
1,Atlanta Hawks_11,46.5 44 U
2,Atlanta Hawks_13,42.5 44 O
3,Atlanta Hawks_14,39.5 38 U
4,Atlanta Hawks_15,42.5 60 O


### Scrape 2000-2009 data

In [5]:
url = 'https://www.sportsoddshistory.com/nba-regular-season-win-total-results-by-team-2000s/'
info = pd.read_html(url)
df = pd.DataFrame(info[0])
df.drop(columns = ['00-01', "2000's Record (Over-Under-Push)"], inplace = True) #drop 00-01 due to lack of data
df.head()

,Team,99-00,01-02,02-03,03-04,04-05,05-06,06-07,07-08,08-09
0,Atlanta Hawks,46.5 28 U,36.5 33 U,41.5 35 U,33 28 U,27.5 13 U,21.5 26 O,28 30 O,38.5 37 U,36.5 47 O
1,Boston Celtics,36.5 35 U,35.5 49 O,47.5 44 U,43.5 36 U,40.5 45 O,40.5 33 U,35 24 U,49.5 66 O,53.5 62 O
2,Brooklyn Nets,40.5 31 U,35.5 52 O,52.5 49 U,51.5 47 U,37.5 42 O,47 49 O,46.5 41 U,43.5 34 U,27.5 34 O
3,Charlotte Hornets,49.5 49 U,45.5 44 U,NaN,NaN,16.5 18 O,21 26 O,32 33 O,35.5 32 U,36.5 35 U
4,Chicago Bulls,27.5 17 U,18.5 21 O,30.5 30 U,36.5 23 U,32.5 47 O,43 41 U,48 49 O,50.5 33 U,40.5 41 O


In [6]:
lst2 = []
for i in range(df.shape[0]-1):
    for col in df.drop(columns = 'Team').columns:
        lst2.append([f'{df.iloc[i][0]}_{col[-2:]}', df[col][i]]) 

df2 = pd.DataFrame(lst2)
df2.rename(columns = {0: 'Team'}, inplace = True)
df2.head()

,Team,1
0,Atlanta Hawks_00,46.5 28 U
1,Atlanta Hawks_02,36.5 33 U
2,Atlanta Hawks_03,41.5 35 U
3,Atlanta Hawks_04,33 28 U
4,Atlanta Hawks_05,27.5 13 U


### Merge data from 2000-2009 with 2010-2019

In [7]:
df1.shape, df2.shape

((261, 2), (261, 2))

In [8]:
df = pd.concat([df2, df1])
df.shape

(522, 2)

In [9]:
df.shape

(522, 2)

In [10]:
df.dropna(inplace = True)

In [11]:
df['win_total'] = df[1].map(lambda x: x.split()[0])
df['actual_wins'] = df[1].map(lambda x: x.split()[1])
df['result'] = df[1].map(lambda x: x.split()[2])
df.drop(columns = 1, inplace=True)
df.head()

,Team,win_total,actual_wins,result
0,Atlanta Hawks_00,46.5,28,U
1,Atlanta Hawks_02,36.5,33,U
2,Atlanta Hawks_03,41.5,35,U
3,Atlanta Hawks_04,33,28,U
4,Atlanta Hawks_05,27.5,13,U


In [12]:
df.shape

(518, 4)

In [13]:
df.loc[df.Team == 'Boston Celtics_18-19']

,Team,win_total,actual_wins,result


### Scrape 20-21, and 21-22 from basketball-reference

In [14]:
url = 'https://www.basketball-reference.com/leagues/NBA_2021_preseason_odds.html'
info = pd.read_html(url)
df_21 = pd.DataFrame(info[0])
df_21.drop(columns = ['Unnamed: 2', 'Odds'], inplace = True)
df_21.rename(columns = {'W-L O/U': 'win_total'}, inplace = True)
df_21.head()

df_21['actual_wins'] = df_21['Result'].map(lambda x: x.split()[0][:2])
df_21['result'] = df_21['Result'].map(lambda x: x.split()[1][1:2].upper())
df_21.drop(columns = 'Result', inplace = True)
df_21['Team'] = df_21['Team'] + '_' + '21'
df_21.head()

,Team,win_total,actual_wins,result
0,Los Angeles Lakers_21,47.5,42,U
1,Milwaukee Bucks_21,49.5,46,U
2,Brooklyn Nets_21,45.5,48,O
3,Los Angeles Clippers_21,46.5,47,O
4,Boston Celtics_21,44.5,36,U


In [15]:
url = 'https://www.basketball-reference.com/leagues/NBA_2022_preseason_odds.html'
info = pd.read_html(url)
df_22 = pd.DataFrame(info[0])
df_22.drop(columns = ['Unnamed: 2', 'Odds'], inplace = True)
df_22.rename(columns = {'W-L O/U': 'win_total'}, inplace = True)
df_22['actual_wins'] = df_22['Result'].map(lambda x: x.split()[0][:2])
df_22['result'] = df_22['Result'].map(lambda x: x.split()[1][1:2].upper())
df_22.drop(columns = 'Result', inplace = True)
df_22['Team'] = df_22['Team'] + '_' + '22'
df_22.head()

,Team,win_total,actual_wins,result
0,Brooklyn Nets_22,56.5,44,U
1,Los Angeles Lakers_22,52.5,33,U
2,Milwaukee Bucks_22,54.5,51,U
3,Golden State Warriors_22,48.5,53,O
4,Utah Jazz_22,52.5,49,U


In [16]:
df_21.shape, df_22.shape

((30, 4), (30, 4))

In [17]:
df_21_22 = pd.concat([df_21,df_22])
df_21_22.shape

(60, 4)

### Concat everything together and save to `.csv`

In [18]:
df_all = pd.concat([df,df_21_22])
df_all.shape

(578, 4)

In [20]:
df_all.to_csv('historical_over_under_data.csv', index = False)